In [ ]:
import requests

url = "https://raw.githubusercontent.com/<username>/<repo>/<branch>/<path_to_file>.py"
response = requests.get(url)

with open("module_name.py", "w") as f:
    f.write(response.text)
